In [255]:
!pip install gradio
!pip install langchain
!pip install pypdf
!pip install selenium Pillow
!pip install pdf2image

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [256]:
import gradio as gr
import re
import feedparser
import requests

def download_and_save_pdf(url):
    arxiv_id = arxiv_id_from_url(url)
    if arxiv_id:
        try:
            # Make a request to the arXiv API
            feed = feedparser.parse(f'http://export.arxiv.org/api/query?id_list={arxiv_id}')

            # Check if the response contains entries
            if 'entries' in feed:
                # Iterate over each entry (paper) in the feed
                for entry in feed.entries:
                    # Extract the PDF link from the entry
                    pdf_link = entry.link.replace('/abs/', '/pdf/') + '.pdf'

                    # Download the PDF
                    response = requests.get(pdf_link)

                    # Save the PDF in the local directory with the name based on the arXiv ID
                    with open(f'{arxiv_id}.pdf', 'wb') as pdf_file:
                        pdf_file.write(response.content)

                    return f"PDF downloaded and saved as {arxiv_id}.pdf"

            else:
                return f"No entries found for arXiv ID {arxiv_id}"

        except Exception as e:
            return f"Error extracting information: {e}"
    else:
        return "Invalid arXiv PDF URL format. Please enter a valid URL."

def arxiv_id_from_url(url):
    # Extract the arXiv ID from the URL using a regular expression
    match = re.search(r'arxiv\.org/pdf/(\d+\.\d+)', url)
    if match:
        return match.group(1)
    else:
        return None


css = """
.container {
    width: 40%;
}
"""



with gr.Blocks(css=css) as demo:
    with gr.Column(elem_classes=["container"]):
        # elem_classes=["container"],
        fn=download_and_save_pdf,
        inputs=gr.Textbox(placeholder="Enter arXiv PDF URL"),
        outputs="text",
        theme="sky"

iface.launch(share=True)
# https://61cc1e9bf3c8b6ff21.gradio.live

Running on local URL:  http://127.0.0.1:7866
Running on public URL: https://61cc1e9bf3c8b6ff21.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


KeyboardInterrupt: 

In [ ]:
iface.close()

In [ ]:
from milvus import default_server
from pymilvus import connections, utility
default_server.start()
connections.connect(host="127.0.0.1", port=default_server.listen_port)
port=default_server.listen_port
host="127.0.0.1"
my_uri = "http://localhost:" + str(port)
print(my_uri)

In [ ]:
import torch
from sentence_transformers import SentenceTransformer

# Initialize torch settings
torch.backends.cudnn.deterministic = True
DEVICE = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
print(f"device: {DEVICE}")

# Load the model from huggingface model hub.
# python -m pip install -U angle-emb
model_name = "WhereIsAI/UAE-Large-V1"
encoder = SentenceTransformer(model_name, device=DEVICE)
print(type(encoder))
print(encoder)

# Get the model parameters and save for later.
EMBEDDING_DIM = encoder.get_sentence_embedding_dimension()
MAX_SEQ_LENGTH_IN_TOKENS = encoder.get_max_seq_length() 
# # Assume tokens are 3 characters long.
# MAX_SEQ_LENGTH = MAX_SEQ_LENGTH_IN_TOKENS * 3
# HF_EOS_TOKEN_LENGTH = 1 * 3
# Test with 512 sequence length.
MAX_SEQ_LENGTH = MAX_SEQ_LENGTH_IN_TOKENS
HF_EOS_TOKEN_LENGTH = 1

# Inspect model parameters.
print(f"model_name: {model_name}")
print(f"EMBEDDING_DIM: {EMBEDDING_DIM}")
print(f"MAX_SEQ_LENGTH: {MAX_SEQ_LENGTH}")

In [ ]:
from langchain.chains.summarize import load_summarize_chain
# from langchain_community.document_loaders import WebBaseLoader
from langchain.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

def get_env_variables():
    """Fetch all necessary configurations from environment variables."""
    return {
        'OPENAI_API_KEY': os.getenv('OPENAI_API_KEY')
    }

load_dotenv()
key = get_env_variables()

loader = PyPDFLoader("1808.05092.pdf")
docs = loader.load()

num_documents = len(docs)
print(f"loaded {num_documents} documents")

In [ ]:
from pymilvus import MilvusClient

# Set the Milvus collection name.
COLLECTION_NAME = "MilvusDocs"

# Add custom HNSW search index to the collection.
# M = max number graph connections per layer. Large M = denser graph.
# Choice of M: 4~64, larger M for larger data and larger embedding lengths.
M = 16
# efConstruction = num_candidate_nearest_neighbors per layer. 
# Use Rule of thumb: int. 8~512, efConstruction = M * 2.
efConstruction = M * 2
# Create the search index for local Milvus server.
INDEX_PARAMS = dict({
    'M': M,               
    "efConstruction": efConstruction })
index_params = {
    "index_type": "HNSW", 
    "metric_type": "COSINE", 
    "params": INDEX_PARAMS
    }

# Use no-schema Milvus client uses flexible json key:value format.
# https://milvus.io/docs/using_milvusclient.md
mc = MilvusClient(
    uri=my_uri # CLUSTER_ENDPOINT,
    # API key or a colon-separated cluster username and password
    # token=TOKEN)
)

# Check if collection already exists, if so drop it.
has = utility.has_collection(COLLECTION_NAME)
if has:
    drop_result = utility.drop_collection(COLLECTION_NAME)
    print(f"Successfully dropped collection: `{COLLECTION_NAME}`")

# Create the collection.
mc.create_collection(COLLECTION_NAME, 
                     EMBEDDING_DIM,
                     consistency_level="Eventually", 
                     auto_id=True,  
                     overwrite=True,
                     # skip setting params below, if using AUTOINDEX
                     params=index_params
                    )

print(f"Successfully created collection: `{COLLECTION_NAME}`")
print(mc.describe_collection(COLLECTION_NAME))

In [ ]:
from langchain.text_splitter import HTMLHeaderTextSplitter, RecursiveCharacterTextSplitter
from bs4 import BeautifulSoup
import numpy as np
import time

start_time = time.time()
# Use the embedding model parameters.
chunk_size = MAX_SEQ_LENGTH - HF_EOS_TOKEN_LENGTH
chunk_overlap = np.round(chunk_size * 0.10, 0)
print(f"chunk_size: {chunk_size}, chunk_overlap: {chunk_overlap}")

# Create an instance of the RecursiveCharacterTextSplitter
child_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_size,
    chunk_overlap = chunk_overlap,
    length_function = len,
)

# Split the documents further into smaller, recursive chunks.
chunks = child_splitter.split_documents(docs)

end_time = time.time()
print(f"chunking time: {end_time - start_time}")
print(f"split into chunks: {len(chunks)}, type: list of {type(chunks[0])}") 

# Inspect a chunk.
print()
print("Looking at a sample chunk...")
print(chunks[0].page_content[:100])
print(chunks[0].metadata)

# # TODO - Uncomment to print child splits with their associated header metadata.
# print()
for child in chunks:
    print(f"Content: {child.page_content}")
    print(f"Metadata: {child.metadata}")
#     print()

In [ ]:
# Clean up the metadata urls
for doc in chunks:
    new_url = doc.metadata["source"]
    new_url = new_url.replace("rtdocs", "https:/")
    doc.metadata.update({"source": new_url})

print(chunks[0].page_content[:100])
print(chunks[0].metadata)

In [ ]:
# Convert chunks to a list of dictionaries.
chunk_list = []
for chunk in chunks:

    # Generate embeddings using encoder from HuggingFace.
    embeddings = torch.tensor(encoder.encode([chunk.page_content]))
    # embeddings = F.normalize(embeddings, p=2, dim=1) #use torch
    embeddings = np.array(embeddings / np.linalg.norm(embeddings)) #use numpy
    converted_values = list(map(np.float32, embeddings))[0]
    
    # Assemble embedding vector, original text chunk, metadata.
    chunk_dict = {
        'vector': converted_values,
        'chunk': chunk.page_content,
        'source': chunk.metadata['source']
    }
    chunk_list.append(chunk_dict)

# Insert data into the Milvus collection.
print("Start inserting entities")
start_time = time.time()
insert_result = mc.insert(
    COLLECTION_NAME,
    data=chunk_list,
    progress_bar=True)
end_time = time.time()
print(f"Milvus Client insert time for {len(chunk_list)} vectors: {end_time - start_time} seconds")

# After final entity is inserted, call flush to stop growing segments left in memory.
mc.flush(COLLECTION_NAME)

In [ ]:
chunk_list[0]

In [ ]:
# Free option

def client_assemble_retrieved_context(retrieved_top_k, metadata_fields=[], num_shot_answers=3):
    """ 
    For each question, assemble the context and metadata from the retrieved_top_k chunks.
    retrieved_top_k: list of dicts
    """
    # Assemble the context as a stuffed string.
    distances = []
    context = []
    context_metadata = []
    i = 1
    for r in retrieved_top_k[0]:
        distances.append(r['distance'])
        if i <= num_shot_answers:
            if len(metadata_fields) > 0:
                metadata = {}
                for field in metadata_fields:
                    metadata[field] = r['entity'][field]
                context_metadata.append(metadata)
            context.append(r['entity']['chunk'])
        i += 1

    # Assemble formatted results in a zipped list.
    formatted_results = list(zip(distances, context, context_metadata))
    # Return all the things for convenience.
    return formatted_results, context, context_metadata

SAMPLE_QUESTION = 'what are the parameters of the experiment?'

# Embed the question using the same encoder.
query_embeddings = encoder.encode([SAMPLE_QUESTION])
TOP_K = 3

# Return top k results with HNSW index.
SEARCH_PARAMS = dict({
    # Re-use index param for num_candidate_nearest_neighbors.
    "ef": INDEX_PARAMS['efConstruction']
    })

# Define output fields to return.
OUTPUT_FIELDS = ["source", "chunk"]

# Embed the question using the same encoder.
query_embeddings = encoder.encode([SAMPLE_QUESTION])
TOP_K = 3

results = mc.search(
    COLLECTION_NAME,
    data=query_embeddings, 
    search_params=SEARCH_PARAMS,
    output_fields=OUTPUT_FIELDS, 
    # Milvus can utilize metadata in boolean expressions to filter search.
    # filter="",
    limit=TOP_K,
    consistency_level="Eventually"
    )

# Assemble `num_shot_answers` retrieved 1st context and context metadata.
METADATA_FIELDS = [f for f in OUTPUT_FIELDS if f != 'chunk']
# formatted_results, context, context_metadata = _utils.client_assemble_retrieved_context(
formatted_results, context, context_metadata = client_assemble_retrieved_context(
    results, metadata_fields=METADATA_FIELDS, num_shot_answers=3)
print(f"Length context: {len(context[0])}, Number of contexts: {len(context)}")

# TODO - Uncomment to loop through each context and metadata and print.
for i in range(len(context)):
    print(f"Retrieved result #{i+1}")
    print(f"Context: {context[i][:150]}")
    print(f"Metadata: {context_metadata[i]}")
    print()

In [ ]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

LLM_NAME = "gpt-3.5-turbo"
TEMPERATURE = 0.1

llm = ChatOpenAI(api_key=os.environ.get("OPENAI_API_KEY"), temperature=TEMPERATURE, model_name=LLM_NAME)

# Define prompt
prompt_template = """
You will be provided a research paper and your task is to summarize the research paper into a 5 minute video as follows:
- Outline the key points of the paper
- Edit the outline into a voiceover script for a 5 minute video
- Clearly state why was the research done, what are the technologies that were previously known involved,
how is the technique or actions performed advancing the field, what are the key metrics that define the success of the work 
and what are future directions that lie ahead.
- Do not write any fact which is not present in the paper
- The final script should contain 4000 words

- First, pretend you are a research scientist who has won accolates for being able to explain expert information to a high-schooler and is giving your dissertation defense.
- Write a clearly organized and to-the-point outline summary of the following research:
"{text}",
- The outline should have 3000 words and objectives should be clearly defined for each section of the paper while preserving the specifics address in the technology used or methods tried that have advanced the particular field.
- Introduce the research scientists involved and the institutions involved if known.
- Every single line in the outline should be in complete sentences, talk with dignity and sophistication. 
- Use phrases such as "Our research presents", "This paper details the", do not use words such as realm, or start the sentence with "In the"
- Assume the audience is asking why and how about the reasoning and logic of the content. 
- Use present tense and do not use past tense.
- Do not use phrases such as "x has been discussed, x has been highlighted", be as specific on the details as possible.
- Make sure to answer clearly what is the major contribution of this body of work.
- The outline should answer to the point and in specific detail why was the research done, what are the technologies that were previously known involved,
how is the technique or actions performed advancing the field, what are the key metrics that define the success of the work 
and what are future directions that lie ahead.

- After you have produced the outline, next convert each point in the outline to be one or more complete sentences in third person point of view, going into detail especially
- regarding the technicalities and key concepts of the research. Make sure that it is absolutely clear in specific detail why was the research done, what are the technologies that were previously known involved,
how is the technique or actions performed advancing the field, what are the key metrics that define the success of the work 
and what are future directions that lie ahead.
- Assume the role of the editor of the best ranking tv production company in the world. 
- Format into a script but not screenplay to be broadcasted publicly in a 5 minute production of 4000 words for higher education consumption.
- Introduce yourself to assume the role of a third party and do not assume the time of day, do not say good evening you are not the researcher but you represent
the researcher in advocating for their work. Provide the narration only, do not format as a screenplay.
Spend at least 6 sentences delving deep into the research key findings and evaluation.

- Lastly edit the entire script to make sure that it is obviously stated to the video viewer why was the research done, what are the technologies that were previously known involved,
how is the technique or actions performed advancing the field, what are the key metrics that define the success of the work 
and what are future directions that lie ahead."""
prompt = PromptTemplate.from_template(prompt_template)

llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

docs = loader.load()
print(stuff_chain.run(docs))

In [ ]:
stuff_chain.run(docs)[:5]

In [ ]:
import requests
import os
from datetime import datetime

def text_to_speech(text_for_TTS):
    ELEVEN_LABS_API_KEY = os.environ.get("ELEVEN_LABS_API_KEY")

    CHUNK_SIZE = 1024
    url = "https://api.elevenlabs.io/v1/text-to-speech/bVMeCyTHy58xNoL34h3p"

    headers = {
        "Accept": "audio/mpeg",
        "Content-Type": "application/json",
        "xi-api-key": ELEVEN_LABS_API_KEY
    }

    data = {
        "text": text_for_TTS,
        "model_id": "eleven_monolingual_v1",
        "voice_settings": {
            "stability": 0.5,
            "similarity_boost": 0.5
        }
    }

    response = requests.post(url, json=data, headers=headers)

    if response.status_code == 200:
        # Generate a unique filename based on timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f'output_{timestamp}.mp3'

        # Save the recording to the unique file
        with open(filename, 'wb') as f:
            for chunk in response.iter_content(chunk_size=CHUNK_SIZE):
                if chunk:
                    f.write(chunk)

        print(f"Recording saved in {filename}")
    else:
        print(f"Error: {response.status_code} - {response.text}")


text_to_speech(stuff_chain.run(docs)[:200])


In [ ]:
import os
from pdf2image import convert_from_path

def convert_pdf_to_png(pdf_path):
    # Create a folder for storing the PNGs
    folder_name = os.path.splitext(os.path.basename(pdf_path))[0] + "_pngs"
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    # Convert each page of the PDF to PNG
    images = convert_from_path(pdf_path, output_folder=folder_name)

    # Save each image as a separate PNG file
    for i, image in enumerate(images):
        png_path = os.path.join(folder_name, f"{folder_name}_page_{i + 1}.png")
        image.save(png_path, "PNG")

    print(f"All pages converted and saved in the folder: {folder_name}")

    # Clean up: Delete the .ppm files
    for filename in os.listdir(folder_name):
        if filename.endswith(".ppm"):
            ppm_path = os.path.join(folder_name, filename)
            os.remove(ppm_path)

    print(f".ppm files deleted in the folder: {folder_name}")

# Example usage:
pdf_path = "1808.05092.pdf"
convert_pdf_to_png(pdf_path)


In [ ]:
from openai import OpenAI
import base64


# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

def generate_image_responses(image_folder):
    # Initialize the OpenAI client
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

    # List to store messages for the OpenAI API call
    messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "If the image has a diagram or visual, output the file name in a list format and whether it is at the top or bottom of the page?",
        },
              ]
    }
    ]

    # Iterate through the images in the folder
    for image_filename in os.listdir(image_folder):
        if image_filename.endswith((".jpg", ".jpeg", ".png")):
            image_path = f"{image_folder}/{image_filename}"  # Replace your_base_url with the actual base URL
            # Getting the base64 string
            base64_image = encode_image(image_path)
            images={
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}",
                        "detail": "low"},
                }
            # Append the images structure to the content list in the messages dictionary
            messages[0]["content"].append(images)

    # Make the OpenAI API call
    response = client.chat.completions.create(
        model="gpt-4-vision-preview",
        messages=messages,
        max_tokens=100,
    )

    # Print the generated responses
    for choice in response.choices:
        print(choice)

# Example usage:
image_folder = "1808.05092_pngs"
generate_image_responses(image_folder)


In [ ]:
import os
from PIL import Image

def cut_pngs_in_half(directory_path):
    # Ensure the directory path is valid
    if not os.path.exists(directory_path):
        print(f"Error: Directory '{directory_path}' does not exist.")
        return

    # Get a list of all files in the directory
    files = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]

    # Process each file in the directory
    for file_name in files:
        # Check if the file is a PNG
        if file_name.lower().endswith('.png'):
            image_path = os.path.join(directory_path, file_name)

            # Open the image
            with Image.open(image_path) as img:
                # Get the dimensions of the image
                width, height = img.size

                # Cut the image in half (top and bottom)
                top_half = img.crop((0, 0, width, height // 2))
                bottom_half = img.crop((0, height // 2, width, height))

                # Save the top and bottom halves with "_cropped_1" and "_cropped_2" suffixes
                top_half.save(os.path.join(directory_path, f"{os.path.splitext(file_name)[0]}_cropped_1.png"), 'PNG')
                bottom_half.save(os.path.join(directory_path, f"{os.path.splitext(file_name)[0]}_cropped_2.png"), 'PNG')

                print(f"Images saved: {file_name}_cropped_1.png (top) and {file_name}_cropped_2.png (bottom)")


if __name__ == "__main__":
    # Get user input for the directory path
    directory_path = "1808.05092_pngs"

    # Call the function to cut PNGs in half
    cut_pngs_in_half(directory_path)
# output_20240310_030318.mp3

In [ ]:
import os
from pydub import AudioSegment
from moviepy.editor import VideoClip, ImageClip, concatenate_videoclips

def analyze_mp3_length(mp3_path):
    audio = AudioSegment.from_mp3(mp3_path)
    return len(audio) / 1000.0  # Length in seconds

def fetch_cropped_images(image_directory):
    cropped_images = [image for image in os.listdir(image_directory) if image.lower().endswith('.png') and 'cropped' in image.lower()]
    return cropped_images

def create_video(mp3_path, image_directory, output_path, ffmpeg_executable):
    mp3_length = analyze_mp3_length(mp3_path)
    cropped_images = fetch_cropped_images(image_directory)

    if not cropped_images:
        print("No cropped images found.")
        return

    # Calculate duration per image
    duration_per_image = mp3_length / len(cropped_images)

    # Create video clips from images
    clips = [ImageClip(os.path.join(image_directory, image), duration=duration_per_image) for image in cropped_images]

    # Concatenate video clips
    video_clip = concatenate_videoclips(clips, method="compose")

    # Create the 'final_videos' directory if it doesn't exist
    final_videos_directory = "final_videos"
    if not os.path.exists(final_videos_directory):
        os.makedirs(final_videos_directory)

    # Write the final video
    output_path = os.path.join(final_videos_directory, output_path)
    video_clip.write_videofile(output_path, codec="libx264", fps=24, ffmpeg_params=["-codec", "libx264", "-preset", "ultrafast", "-pix_fmt", "yuv420p"])




if __name__ == "__main__":
    # Example usage:
    mp3_path = "output_20240310_030318.mp3"
    image_directory = "1808.05092_pngs"
    output_path = "final_videos/1808.05092.mp4"
    ffmpeg_executable = "/opt/homebrew/bin/ffmpeg"

    create_video(mp3_path, image_directory, output_path, ffmpeg_executable)


In [265]:
!pip install opencv-python

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.4/35.4 MB 9.7 MB/s eta 0:00:00:00:0100:01


In [273]:
import os
import cv2
from pydub import AudioSegment

def analyze_mp3_length(mp3_path):
    audio = AudioSegment.from_file(mp3_path)
    return len(audio) / 1000.0  # Length in seconds

def fetch_cropped_images(image_directory):
    cropped_images = [image for image in os.listdir(image_directory) if image.lower().endswith('.png') and 'cropped' in image.lower()]
    sorted_images = sorted(cropped_images, key=lambda x: int(''.join(filter(str.isdigit, x))))
    return sorted_images

def create_video_with_audio(mp3_path, image_directory):
    mp3_length = analyze_mp3_length(mp3_path)
    cropped_images = fetch_cropped_images(image_directory)

    if not cropped_images:
        print("No cropped images found.")
        return

    # Calculate duration per image
    duration_per_image = mp3_length / len(cropped_images)

    # Initialize video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output_path = "output_video.mp4"  # Define output video path
    video_writer = cv2.VideoWriter(output_path, fourcc, 30, (640, 480))  # Adjust resolution as needed

    for image in cropped_images:
        img = cv2.imread(os.path.join(image_directory, image))
        if img is not None:  # Check if image was successfully read
            resized_img = cv2.resize(img, (640, 480))
            video_writer.write(resized_img)
        else:
            print(f"Error reading {image}")

    video_writer.release()

    # Add audio to the video
    video = cv2.VideoCapture(output_path)
    audio = AudioSegment.from_file(mp3_path)
    
    frame_rate = video.get(cv2.CAP_PROP_FPS)
    
    output_with_audio_path = "final_video_with_audio.mp4"
    
    fourcc_audio = cv2.VideoWriter_fourcc(*'mp4v')  # Change codec to match video codec
    audio_writer = cv2.VideoWriter(output_with_audio_path, fourcc_audio, frame_rate, (640, 480))
    
    while True:
        ret, frame = video.read()
        if not ret:
            break
        
        audio_writer.write(frame)
    
    audio_writer.release()
    
if __name__ == "__main__":
    # Example usage:
    mp3_path = "output_20240310_030318.mp3"
    image_directory = "1808.05092_pngs"
    
    create_video_with_audio(mp3_path, image_directory)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 